In [ ]:
!pip install transformers

In [ ]:
!pip install thundersvm-cuda10

In [ ]:
import pandas as pd

train_data = pd.read_csv('/kaggle/input/authattrdata/train_tweets.txt', sep = '\t', header = None)
#print(train_data.head(5))

In [ ]:
y = train_data[0]
tweets = train_data[1]

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

print('labels:', len(set(y)))
print('samples:', len(tweets))

distt = Counter(y)
counts = []

for key, val in distt.most_common():
    counts.append(val)

plt.plot(counts)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased", do_lower_case = False)
X = []
for tweet in tweets:
    enc = tokenizer.encode_plus(tweet,
                        max_length = 48,
                        padding = 'max_length',
                        truncation = True)
    X.append(enc['input_ids'])

df = pd.DataFrame.from_records(X)
df.to_csv('/kaggle/working/bert_tweets.csv')

In [ ]:
import numpy as np
df = pd.read_csv('/kaggle/working/bert_tweets.csv', dtype = str)

X = []
for val in df.values.tolist():
    X.append(' '.join(val[1:]))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 30000, lowercase = False, sublinear_tf = True)

X = vectorizer.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25,random_state=42)

In [ ]:
from thundersvm import SVC

clf = SVC(max_iter = 10000, kernel = 'linear').fit(X_train, y_train)

In [ ]:
from sklearn.metrics import balanced_accuracy_score

y_pred=clf.predict(X_test)
print('accuracy:', balanced_accuracy_score(y_test, y_pred))